<a href="https://colab.research.google.com/github/sankethm7/ai_ml_lab/blob/main/9_MultiAgent_Customer_Support_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem statement: Build Multi-AI-Agent system for Customer Support system.

2 cusotmer support executive roles
1. Customer Support Representative
Tasks:
* Interacts with the customer, takes query, finds answers, generate response, share to QA specialist, and upon approval, shares the generated answer back to customer

2. Support QA Specialist
Tasks:
* Review answer generated by Customer Support Representative, suggests it to make it better or approve

Tool Framework used: CrewAI

Code modules:
- Define Agents
- Setup Tasks for each agent
- Setup tools to be used
- Configure CrewAI agnet framework
- Execute CrewAI

In [ ]:
!pip install crewai
!pip install crewai_tools
!pip install langchain_community

In [ ]:
# Warning control

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# import classes from crewai and crewai_tools

from crewai import Agent, Task, Crew

from crewai_tools import SerperDevTool,ScrapeWebsiteTool, WebsiteSearchTool

In [ ]:
# setup secret and model

import os
import openai

from google.colab import userdata
#openai.api_key = userdata.get('OPEN_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPEN_API_KEY')
os.environ['OPENAI_MODEL_NAME']= 'gpt-3.5-turbo'
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')


In [ ]:
# setup Role Playing, Focus and Cooperation for Suport Agent

support_agent = Agent(
    role="Support Representative",
    goal="Be the most friendly, knowleded and helpful support representative in the company",
    backstory=("You work at AI ML Etc.  (https://www.aimletc.com/)."
              "You are workinng and providining support to {customer}."
              "You are quick learner for your company."
              "You need to make sure that you provide the best support!"
              "Make sure to provide complete answers, and make no assumptions."),
    allow_delegation=False,
    verbose=True,
)

# setup Role Playing, Focus and Cooperation for Suport QA Agent
support_qa_agent = Agent(
    role="Support QA Specialist",
    goal="Get recognized for providing the best support QA in the company",
    backstory=("You wokr at AI ML Etc. (https://www.aimletc.com/)."
              "You are working with your team on a request from a {customer}"
              "You ensure that the support representative is providing the best support possible. "
              "You need to make sure that the support representative is providing complete answers and making no assumptions."),
    verbose=True,
)

In [ ]:
# Instantiate a document scraper tool.
# The tool will scrape a page (only 1 URL) of the CrewAI documentation.

docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://www.aimletc.com/online-instructor-led-ai-llm-coaching-for-it-technical-professionals/"
)

In [ ]:
# Different Ways to Give Agents Tools
# Agent Level: The Agent can use the Tool(s) on any Task it performs.
# Task Level: The Agent will only use the Tool(s) when performing that specific Task.

# Note: Task Tools override the Agent Tools.

# You are passing the Tool on the Task Level.

# set up the task for support representative
inquiry_resolution = Task(
    description=("{customer} just reached out with a super important ask: {inquiry}.\n\n "
                "{user} from {customer} is the one that reached out."
                "Make sure to use everything you know to provide the best possible support."
                "You must strive to provide a complete and accurate response to the customer's inquiry."
                "Do not disclose nay Personally Indentifiable Information (PII). "
                "Do not take any competitier's name or reference."),

    expected_output=("A detailed, informative respons to the customer's inquiry that addresses all aspects of their question."
                    "The response should include references to everything you used to find the answer."
                    "The response should include external data or solutions."
                    "Ensure the answer is complete, leaving no questions unanswered, and maintain a helpful and friendly tone throughout."),

    agent = support_agent,
)

# set up the task for support qa
supprt_qa_review = Task(
    description=("Review the response drafted by the Suport Representative for {user}'s inquiry."
                "Ensure that the answer is comprehensive, accurate, and adheres to the high-quality standards expected for customer support."
                "verify that all parts of the customer's inquiry have been addressed thoroughly, with a helpful and friendly tone."
                "Check for reference and sources used to find the information, ensuring the response is well-supported and leaves no questions unanswered."),

    expected_output=("A final, detailed and informative response ready to be sent to the customer."
                    "This response should fully address the customer's inquiry, incorporating all relevant feedback and improvements."
                    "Don't be too formal, we are a chill and cool company but maintain a professional and friendly tone throughout."),

    agent = support_qa_agent,
)

In [ ]:
# creatinng a Crew

print(os.environ.get('OPEN_API_KEY'))

if OPENAI_API_KEY is None or OPENAI_API_KEY == '':
    raise ValueError("OPEN_API_KEY is not set. Please provide a valid OpenAI API key.")

crew = Crew(
    agents=[support_agent, support_qa_agent],
    tasks=[inquiry_resolution, supprt_qa_review],
    verbose=True,
    memory=True,
    openai_api_key = OPENAI_API_KEY,
)

None


In [ ]:
# Running the crew

set_inputs = {
    "customer": "My Company",
    "user" : "Sanket",
    "inquiry": "I need help with the courses offered by AI ML Etc. What are the concepts covered in the course?"
}
result = crew.kickoff (inputs = set_inputs)

# Agent: Support Representative
## Task: My Company just reached out with a super important ask: I need help with the courses offered by AI ML Etc. What are the concepts covered in the course?.

 Sanket from My Company is the one that reached out.Make sure to use everything you know to provide the best possible support.You must strive to provide a complete and accurate response to the customer's inquiry.Do not disclose nay Personally Indentifiable Information (PII). Do not take any competitier's name or reference.


# Agent: Support Representative
## Final Answer: 
Thank you for considering AI ML Etc. for your learning needs, Sanket!

Based on your inquiry, our courses cover a wide spectrum of foundational and advanced topics in Artificial Intelligence (AI) and Machine Learning (ML):

1. **Introduction to AI and ML**: You will establish a solid foundation by understanding the basics of AI and ML, distinguishing between the two fields, and exploring their diverse applications in various

In [ ]:
# display final results as Markdown

from IPython.display import Markdown
Markdown(result.raw)

Hi Sanket,

Thank you for considering AI ML Etc. for your learning journey! I'm glad you reached out to us with your inquiry. Our platform offers a diverse range of courses covering various areas within Artificial Intelligence (AI) and Machine Learning (ML). Here's a comprehensive overview of the topics we cover in our courses to help you make an informed decision:

1. **Introduction to AI and ML**: Our courses provide a solid foundation by covering the basics of AI and ML, helping you differentiate between the two fields and showcasing their applications across industries.

2. **Machine Learning Algorithms**: Dive into popular algorithms like regression, classification, clustering, and deep learning to enhance your understanding of how machines learn and predict patterns effectively.

3. **Data Preprocessing and Feature Engineering**: Learn essential steps in data cleaning, preparation, and feature generation to optimize model performance.

4. **Model Evaluation and Hyperparameter Tuning**: Discover effective methods to evaluate model performance and fine-tune hyperparameters for better ML project outcomes.

5. **Natural Language Processing (NLP)**: Explore advanced NLP techniques such as sentiment analysis and chatbot applications for processing human language data.

6. **Computer Vision**: Gain insights into computer vision concepts for image recognition, detection, and segmentation and develop skills in training models with visual data.

7. **Reinforcement Learning**: Delve into reinforcement learning principles and applications to train agents for sequential decision-making.

8. **Ethical Implications of AI**: Engage in discussions on ethical considerations essential for responsible AI system development, emphasizing fair and unbiased practices in projects.

Our courses combine theoretical knowledge with practical applications through hands-on projects and real-world examples. In addition to course materials, we offer resources like tutorials, case studies, and expert support to enhance your learning experience.

For more detailed information on specific courses and their curriculum, feel free to explore our website. If you have any more questions or need assistance, our dedicated support team is here to help you achieve your learning goals.

Warm regards,
[Your Name]
Support Team AI ML Etc.